In [6]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter


loader = TextLoader("speech.txt", encoding="utf8")
documents= loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
docs = text_splitter.split_documents(documents)

In [7]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='“The Power of Curiosity”\n\nGood [morning/afternoon/evening] everyone,\n\nHave you ever wondered why we keep asking questions, even when answers seem just out of reach? Curiosity is not just a spark in our minds—it’s the engine of progress. Every great invention, every bold idea, every story of triumph started with a simple question: “What if…?”\n\nCuriosity pushes us beyond comfort, beyond routine, and into discovery. It encourages us to fail, to stumble, and then to rise stronger. It reminds us that knowledge is not a destination, but a journey.\n\nSo today, I urge each of you: embrace your curiosity. Ask questions that scare you. Explore paths no one has walked before. Because in the act of seeking, in the courage to wonder, we find our greatest potential.\n\nRemember, the world doesn’t change because we play it safe—it changes because we dared to ask, dared to learn, and dared to act.\n\nThank you.')]

In [9]:
embeddings=OllamaEmbeddings()
db = FAISS.from_documents(docs, embeddings)
db

In [10]:
### querying 

query = " what is the main topic discussed in the speech?"


In [13]:
docs = db.similarity_search(query)
docs[0].page_content

'“The Power of Curiosity”\n\nGood [morning/afternoon/evening] everyone,\n\nHave you ever wondered why we keep asking questions, even when answers seem just out of reach? Curiosity is not just a spark in our minds—it’s the engine of progress. Every great invention, every bold idea, every story of triumph started with a simple question: “What if…?”\n\nCuriosity pushes us beyond comfort, beyond routine, and into discovery. It encourages us to fail, to stumble, and then to rise stronger. It reminds us that knowledge is not a destination, but a journey.\n\nSo today, I urge each of you: embrace your curiosity. Ask questions that scare you. Explore paths no one has walked before. Because in the act of seeking, in the courage to wonder, we find our greatest potential.\n\nRemember, the world doesn’t change because we play it safe—it changes because we dared to ask, dared to learn, and dared to act.\n\nThank you.'

## as a Retiever

we can also convert the vectorstore into a Retriever class . This allows us to easily use it in other langchain methods, which largely work with retrievers.


In [15]:
retriever = db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'“The Power of Curiosity”\n\nGood [morning/afternoon/evening] everyone,\n\nHave you ever wondered why we keep asking questions, even when answers seem just out of reach? Curiosity is not just a spark in our minds—it’s the engine of progress. Every great invention, every bold idea, every story of triumph started with a simple question: “What if…?”\n\nCuriosity pushes us beyond comfort, beyond routine, and into discovery. It encourages us to fail, to stumble, and then to rise stronger. It reminds us that knowledge is not a destination, but a journey.\n\nSo today, I urge each of you: embrace your curiosity. Ask questions that scare you. Explore paths no one has walked before. Because in the act of seeking, in the courage to wonder, we find our greatest potential.\n\nRemember, the world doesn’t change because we play it safe—it changes because we dared to ask, dared to learn, and dared to act.\n\nThank you.'

### Similarity Search with score 

there are some FAISS specific methods we can use to get more information about the search results. For example, we can get the similarity score for each document retrieved.

In [16]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score[0][0].page_content, docs_and_score[0][1]

('“The Power of Curiosity”\n\nGood [morning/afternoon/evening] everyone,\n\nHave you ever wondered why we keep asking questions, even when answers seem just out of reach? Curiosity is not just a spark in our minds—it’s the engine of progress. Every great invention, every bold idea, every story of triumph started with a simple question: “What if…?”\n\nCuriosity pushes us beyond comfort, beyond routine, and into discovery. It encourages us to fail, to stumble, and then to rise stronger. It reminds us that knowledge is not a destination, but a journey.\n\nSo today, I urge each of you: embrace your curiosity. Ask questions that scare you. Explore paths no one has walked before. Because in the act of seeking, in the courage to wonder, we find our greatest potential.\n\nRemember, the world doesn’t change because we play it safe—it changes because we dared to ask, dared to learn, and dared to act.\n\nThank you.',
 np.float32(18726.387))

In [17]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[1.711087942123413,
 -1.0305181741714478,
 0.7862368226051331,
 -0.07662911713123322,
 -1.0592323541641235,
 0.8560511469841003,
 0.6834619641304016,
 0.21296389400959015,
 -0.11853352189064026,
 -0.24641934037208557,
 2.502746820449829,
 0.008594796061515808,
 -0.24588483572006226,
 0.5829282999038696,
 -0.13413886725902557,
 -0.5745611190795898,
 -0.3607093393802643,
 0.9547005295753479,
 0.5930108428001404,
 -2.4250857830047607,
 -0.48815664649009705,
 -0.9994722604751587,
 1.9852306842803955,
 -0.9455716013908386,
 1.5977096557617188,
 -0.7162306904792786,
 0.4812014102935791,
 -1.6589010953903198,
 -0.19909623265266418,
 -0.4478388726711273,
 2.235719680786133,
 -2.6371378898620605,
 -2.536475419998169,
 4.6959147453308105,
 3.734531879425049,
 -3.1750316619873047,
 -0.7213644981384277,
 0.9549746513366699,
 -0.49529993534088135,
 -0.7665112614631653,
 -0.451029896736145,
 -5.262860298156738,
 0.678905725479126,
 0.7829099893569946,
 -0.4460599422454834,
 -0.3263871669769287,
 -0.

In [19]:
docs_score=db.similarity_search_by_vector(embedding_vector)

In [20]:
docs_score

[Document(id='fd53df2b-1bf8-4aa1-b501-2e1b20707b1f', metadata={'source': 'speech.txt'}, page_content='“The Power of Curiosity”\n\nGood [morning/afternoon/evening] everyone,\n\nHave you ever wondered why we keep asking questions, even when answers seem just out of reach? Curiosity is not just a spark in our minds—it’s the engine of progress. Every great invention, every bold idea, every story of triumph started with a simple question: “What if…?”\n\nCuriosity pushes us beyond comfort, beyond routine, and into discovery. It encourages us to fail, to stumble, and then to rise stronger. It reminds us that knowledge is not a destination, but a journey.\n\nSo today, I urge each of you: embrace your curiosity. Ask questions that scare you. Explore paths no one has walked before. Because in the act of seeking, in the courage to wonder, we find our greatest potential.\n\nRemember, the world doesn’t change because we play it safe—it changes because we dared to ask, dared to learn, and dared to a

In [21]:
### saving and the loading the FAISS vectorstore
db.save_local("faiss_index")


In [23]:
new_db= FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)

In [24]:
docs = new_db.similarity_search(query)